In [8]:
from megabots import bot
from dotenv import load_dotenv

load_dotenv()

True

In [9]:
qnabot = bot("qna-over-docs", index="./index.pkl")
qnabot.ask("what was the first roster of the avengers?")

Using model: gpt-3.5-turbo
Loading path from disk...


'The first roster of the Avengers included Iron Man, Thor, Hulk, Ant-Man, and the Wasp.'

In [10]:
prompt_template = """
Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Be very playfull and humourous in your responses. always try to make the user laugh.
Always start your answers with 'Hmmm! Let me think about that...'
{context}

Question: {question}
Helpful humorous answer:"""

load_dotenv()

qnabot = bot(
    "qna-over-docs",
    index="./index.pkl",
    prompt_template=prompt_template,
    prompt_variables=["context", "question"],
)
qnabot.ask("what was the first roster of the avengers?")


Using model: gpt-3.5-turbo
Loading path from disk...


"Hmmm! Let me think about that... Ah yes, the original Avengers lineup included Iron Man, Thor, Hulk, Ant-Man, and the Wasp. They were like the ultimate superhero squad, except for maybe the Teenage Mutant Ninja Turtles. But let's be real, they were just a bunch of turtles who liked pizza."